In [1]:
# %pip install requests
# %pip install beautifulsoup4

In [8]:
# Khởi tạo Producer
from kafka import KafkaProducer
from time import sleep 
import json

producer = KafkaProducer(bootstrap_servers = ['localhost:9092'], api_version = (0,10,1))

In [6]:
# Import 
import requests
from bs4 import BeautifulSoup

In [21]:
url = "https://vnexpress.net/apple-ra-mat-iphone-14-4508267.html"

# Gửi yêu cầu GET đến trang web
response = requests.get(url)

# Kiểm tra xem yêu cầu có thành công hay không (status code 200 là thành công)
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích nội dung trang web
    soup = BeautifulSoup(response.content, 'html.parser')

    # Lấy nội dung trong thẻ <title>
    title_tag = soup.find('title')

    # Kiểm tra xem thẻ <title> có tồn tại hay không trước khi lấy nội dung
    if title_tag:
        title = title_tag.text
        title_bytes = str(title).encode('utf-8')
        producer.send('crawl_data_web', title_bytes)
        # print("Tiêu đề:", title)
    else:
        print("Không tìm thấy thẻ <title> trên trang web.")

    # Lấy tất cả các thẻ <a> trên trang web
    all_a_tags = soup.find_all('a')

    # Kiểm tra xem thẻ <a> có tồn tại hay không trước khi lấy nội dung
    if all_a_tags:
        for a_tag in all_a_tags:
            a_text = a_tag.text
            a_bytes = str(a_text).strip().replace("\n", "").encode('utf-8')
            producer.send('crawl_data_web', a_bytes)
    else:
        print("Không tìm thấy thẻ <a> trên trang web.")

else:
    print("Không thể kết nối đến trang web. Mã trạng thái:", response.status_code)

